# Tool Usage

Let's see how to evaluate an agent's ability to use tools.

In [1]:
from langchain_benchmarks import clone_public_dataset, registry

For this code to work, please configure LangSmith environment variables with your credentials.

In [2]:
registry

Name,Type,Dataset ID,Description
Tool Usage - Typewriter (1 func),ToolUsageTask,placeholder,"Environment with a single function that accepts a single letter as input, and ""prints"" it on a piece of paper. The objective of this task is to evaluate the ability to use the provided tools to repeat a given input string. For example, if the string is 'abc', the tools 'a', 'b', and 'c' must be invoked in that order. The dataset includes examples of varying difficulty. The difficulty is measured by the length of the string."
Tool Usage - Typewriter,ToolUsageTask,placeholder,"Environment with 26 functions each representing a letter of the alphabet. In this variation of the typewriter task, there are 26 parameterless functions, where each function represents a letter of the alphabet (instead of a single function that takes a letter as an argument). The object is to evaluate the ability of use the functions to repeat the given string. For example, if the string is 'abc', the tools 'a', 'b', and 'c' must be invoked in that order. The dataset includes examples of varying difficulty. The difficulty is measured by the length of the string."
Tool Usage - Relational Data,ToolUsageTask,e95d45da-aaa3-44b3-ba2b-7c15ff6e46f5,"Environment with fake data about users and their locations and favorite foods. The environment provides a set of tools that can be used to query the data. The objective of this task is to evaluate the ability to use the provided tools to answer questions about relational data. The dataset contains 21 examples of varying difficulty. The difficulty is measured by the number of tools that need to be used to answer the question. Each example is composed of a question, a reference answer, and information about the sequence in which tools should be used to answer the question. Success is measured by the ability to answer the question correctly, and efficiently."
Multiverse Math,ToolUsageTask,placeholder,"An environment that contains a few basic math operations, but with altered results. For example, multiplication of 5*3 will be re-interpreted as 5*3*1.1. The basic operations retain some basic properties, such as commutativity, associativity, and distributivity; however, the results are different than expected. The objective of this task is to evaluate the ability to use the provided tools to solve simple math questions and ignore any innate knowledge about math."
Email Extraction,ExtractionTask,https://smith.langchain.com/public/36bdfe7d-3cd1-4b36-b957-d12d95810a2b/d,"A dataset of 42 real emails deduped from a spam folder, with semantic HTML tags removed, as well as a script for initial extraction and formatting of other emails from an arbitrary .mbox file like the one exported by Gmail. Some additional cleanup of the data was done by hand after the initial pass. See https://github.com/jacoblee93/oss-model-extraction-evals."


In [3]:
task = registry["Tool Usage - Relational Data"]
task

Name,Tool Usage - Relational Data
Type,ToolUsageTask
Dataset ID,e95d45da-aaa3-44b3-ba2b-7c15ff6e46f5
Description,Environment with fake data about users and their locations and favorite foods. The environment prov...


In [4]:
print(task.description)

Environment with fake data about users and their locations and favorite foods.

The environment provides a set of tools that can be used to query the data.

The objective of this task is to evaluate the ability to use the provided tools to answer questions about relational data.

The dataset contains 21 examples of varying difficulty. The difficulty is measured by the number of tools that need to be used to answer the question.

Each example is composed of a question, a reference answer, and information about the sequence in which tools should be used to answer the question.

Success is measured by the ability to answer the question correctly, and efficiently.



Clone the dataset associaetd with this task

In [5]:
clone_public_dataset(task.dataset_id, dataset_name=task.name)

Dataset Tool Usage - Relational Data already exists. Skipping.
You can access the dataset at https://smith.langchain.com/o/e081f11e-fbd2-41b4-9fa8-5d76c76ef854/datasets/f2b5a831-8eef-4bc7-b6de-68078b87350f.


## Define an agent

Let's build an agent that we can use for evaluation.

In [9]:
from langchain_benchmarks.tool_usage import agents

In [10]:
agent_factory = agents.OpenAIAgentFactory(task, model="gpt-3.5-turbo-16k")

Let's test that our agent works

In [11]:
agent = agent_factory.create()

In [12]:
agent.invoke({"question": "who is bob?"})

{'question': 'who is bob?',
 'output': 'Bob is a user with the name "Bob".',
 'intermediate_steps': [(AgentActionMessageLog(tool='find_users_by_name', tool_input={'name': 'bob'}, log="\nInvoking: `find_users_by_name` with `{'name': 'bob'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "name": "bob"\n}', 'name': 'find_users_by_name'}})]),
   [{'id': 21, 'name': 'Bob'},
    {'id': 41, 'name': 'Donna'},
    {'id': 1, 'name': 'Alice'},
    {'id': 35, 'name': 'Charlie'},
    {'id': 42, 'name': 'Eve'},
    {'id': 43, 'name': 'Frank The Cat'}]),
  (AgentActionMessageLog(tool='get_user_name', tool_input={'user_id': 21}, log="\nInvoking: `get_user_name` with `{'user_id': 21}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "user_id": 21\n}', 'name': 'get_user_name'}})]),
   'Bob')]}

## Eval

Let's evaluate an agent now

In [13]:
from langchain_benchmarks.tool_usage import STANDARD_AGENT_EVALUATOR
from langsmith.client import Client

In [14]:
client = Client()

In [ ]:
test_run = client.run_on_dataset(
    dataset_name=task.name,
    llm_or_chain_factory=agent_factory.create,
    evaluation=STANDARD_AGENT_EVALUATOR,
    verbose=True,
    tags=["openai-functions"],
)